This Jupyter notebook will focus on creating Dashboard using Plotly and Dash  
Components of Dashboard will be as follows:  
FIRST: Yearly automobile sales statistics  
Yearly automobile sales using line chart for whole period  
For choosen year provide:  
Total monthly Automobile sales using line chart  
Average monthly Automobile sales using bar chart  
Total advertisment expenditure for each vehicle using pie chart  
SECOND: Recession Period Statistics  
Average automobile sales using line chart for Recession Period  
Average number of vehicles sold by vehicle type using bar chart  
Total expenditure share by vehicle type during recession using bar chart  
Effect of unemployment rate on vehicle type and sales using bar chart

In [81]:
import pandas as pd
import numpy as np
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
year_list =  [i for i in range(1980, 2024, 1)]
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
df = pd.read_csv(URL)
df.head()

,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City
0,1/31/1980,1980,Jan,1,108.24,0.50,27483.571,1558,7,60.223,0.010000,5.4,456.0,Supperminicar,Georgia
1,2/29/1980,1980,Feb,1,98.75,0.75,24308.678,3048,4,45.986,-0.309594,4.8,555.9,Supperminicar,New York
2,3/31/1980,1980,Mar,1,107.48,0.20,28238.443,3137,3,35.141,-0.308614,3.4,620.0,Mediumfamilycar,New York
3,4/30/1980,1980,Apr,1,115.01,1.00,32615.149,1653,7,45.673,0.230596,4.2,702.8,Supperminicar,Illinois
4,5/31/1980,1980,May,1,98.72,0.20,23829.233,1319,4,52.997,0.138197,5.3,770.4,Smallfamiliycar,California


In [106]:
# Define a Dash application
app = dash.Dash(__name__)

# Defining layout

app.layout = html.Div(
    children =[
        html.H1('Automobile Sales Statistics Dashboard', style={'text-align':'center','color':'#503D36','font-size': 24}),
        #adding a div containing dropdown
        html.Div( [
            dcc.Dropdown(id='dropdown-statistics',
                         options=[{'label':'Yearly Statistics', 'value':'Yearly Statistics'},{'label':'Recession Period Statistics','value':'Recession Period Statistics'}],
                        placeholder='Select a report type',
                         value='Select Statistics',
                        style={'width':'80','padding':'3px','font-size':'20px','text-align':'center'}),
            
            dcc.Dropdown(id = 'select-year',
                         options= [{'label': i, 'value': i} for i in year_list],
                         placeholder ='Select year',
                         value='select-year',
                         style={'width':'80','padding':'3px','font-size':'20px','text-align':'center'}
                
            )
        ]
        ),
        # now adding a division for out put display
        html.Div(
            [html.Div(id='output-container', className='chart-grid', style={'display': 'flex'}),]
            
        )
        
    ]
)

# Next step is to create a callback decorator which will be connecting the input and output components
# Here we will need two callback decorators
# First define a callback to update the input container based on selected statistics
@app.callback(
             Output(component_id='select-year', component_property='disabled'),
             Input(component_id='dropdown-statistics', component_property='value'))



#defining function
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True
        
        
# Second define callback for plotting graphs

@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='select-year', component_property='value')])


def update_output_container(selected_statistics, input_year):
    
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = df[df['Recession'] == 1]

        # Next step is to create and display graphs for Recession Period Statistics
        
        
        # Plot one: Automobile sales fluctuate over the years
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure = px.line(yearly_rec, x='Year',y='Automobile_Sales', title='How Average Automobile Sales fluctuate over the Recession years')
        )

        
        # Plot 2: Calculate average number of vehicles sold by vehicle type
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure= px.bar(average_sales,x='Vehicle_Type', y='Automobile_Sales',title="Average Number of Vehicles Sold by Vehicle Type")
        )

        
        #Plot 3 Pie chart for total expenditure share by vehicle type during recession
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure = px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type',title="Total Expenditure Share by Vehicle Type During Recessions")
        )

        
        # PLot 4 Bar chart for effect of unemployment rate on vehicle type and sales
        unempl = recession_data.groupby(['Vehicle_Type','unemployment_rate'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure = px.bar(unempl,x='Vehicle_Type',y='Automobile_Sales', color='unemployment_rate', title='Effect of Unemployment Rate on Vehicle Type and Sales')
        )
        
        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)], style={'display':'flex'})
            ]

    # Now moving on to second component 
    # creating plots for Yearly data

    
    elif(input_year and selected_statistics == 'Yearly Statistics'):
        yearly_data = data[data['Year']==input_year]

        # Creating graphs
                              
#plot 1 Yearly Automobile sales using line chart for the whole period.
        yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales'))
            
# Plot 2 Total Monthly Automobile sales using line chart.
        monthly_sales = data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(monthly_sales, x= 'Month', y='Automobile_Sales',
            title= 'Total Monthly Automobile Sales'))

            # Plot bar chart for average number of vehicles sold during the given year
        avr_vdata=yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph( figure =px.bar(avr_vdata,x='Year', y='Automobile_Sales',title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year)))

            # Total Advertisement Expenditure for each vehicle using pie chart
        exp_data=yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(exp_data, values = 'Advertising_Expenditure', names='Vehicle_Type'))

#TASK 2.6: Returning the graphs for displaying Yearly data
        return [
            html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': 'flex'})
            ]
        




#NOw running the application
if __name__ == '__main__':
    app.run(port=91)
